In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

**Customer Lifetime Value (CLV)** = how much a company expects to earn from an average customer in a life time.

CLV allows to benchmark customers and identify how much money the company can afford to spend on customer acquisition.

**Historical CLV** = the sum of revenues of all customer transactions multiplied by average or product-level profit margin

Problems with historical CLV:
- doesn't account for customer tenure, retention or churn  (e.g. if the company is growing its customer base, historical CLV will be deflated due to short tenure)
- doesn't account for new customers and their future revenue

**Basic CLV** = Average Revenue * Profit Margin * Average Lifespan <br>
(where Average Lifespan is e.g. average time before customer churn)

**Granular CLV** = (Average Revenue per Transaction * Average Frequency * Profit Margin) * Average Lifespan <br>
(where Avg Frequency is within the certain timeframe, e.g. a month) <br>
accounts for each transaction

**Traditional CLV** = (Average Revenue * Profit Margin) * Retention Rate / Churn Rate <br>
(where Churn = 1 - Retention Rate) <br>
Retention/Churn - a proxy of expected length of customer lifespan with the company <br>
account for customer loyalty
assumes that churn is final and customers do not return (especially critical for non-contractual business models)

Because we don't have profit margin, we will calculate revenue-based CLV. 

But before calculating Basic, Granular and Traditional CLV, we will load, explore the data and then calculate retention rates using cohort analysis. We will need retention rates and churn rates for calculating Traditional CLV later.

## Step 1: Load and Explore Data

In [2]:
retail = pd.read_csv("../cohort_analysis/online_retail_II.csv")

In [3]:
retail.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
retail.shape

(1067371, 8)

In [5]:
print("number of unique customers:", retail['Customer ID'].nunique())

number of unique customers: 5942


In [6]:
#checking duplicates
print(f"there are {retail.duplicated().sum()} duplicated rows")

there are 34335 duplicated rows


In [7]:
#viewing duplicated rows
retail[retail.duplicated()].head(10)

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
371,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
383,489517,22130,PARTY CONE CHRISTMAS DECORATION,6,2009-12-01 11:34:00,0.85,16329.0,United Kingdom
384,489517,22319,HAIRCLIPS FORTIES FABRIC ASSORTED,12,2009-12-01 11:34:00,0.65,16329.0,United Kingdom
385,489517,21913,VINTAGE SEASIDE JIGSAW PUZZLES,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
386,489517,21821,GLITTER STAR GARLAND WITH BELLS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
390,489517,84951A,S/4 PISTACHIO LOVEBIRD COASTERS,1,2009-12-01 11:34:00,2.55,16329.0,United Kingdom
391,489517,21491,SET OF THREE VINTAGE GIFT WRAPS,1,2009-12-01 11:34:00,1.95,16329.0,United Kingdom
394,489517,21912,VINTAGE SNAKES & LADDERS,1,2009-12-01 11:34:00,3.75,16329.0,United Kingdom
657,489529,22028,PENNY FARTHING BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom
658,489529,22036,DINOSAUR BIRTHDAY CARD,12,2009-12-01 11:51:00,0.42,17984.0,United Kingdom


Note: After manual review, there are indeed lots of duplicated rows (e.g. row 371 and 394). Let's remove them.

In [8]:
retail = retail.drop_duplicates(keep='first')

In [9]:
#checking missing values
retail.isna().sum()

Invoice             0
StockCode           0
Description      4275
Quantity            0
InvoiceDate         0
Price               0
Customer ID    235151
Country             0
dtype: int64

Note: there are 234007 rows without Customer ID. Since we are working at the customer level, we cannot aggregate these columns. Let's remove them from further analysis.

In [10]:
retail = retail[retail['Customer ID'].notna()]

In [11]:
# checking types of columns
retail.dtypes

Invoice         object
StockCode       object
Description     object
Quantity         int64
InvoiceDate     object
Price          float64
Customer ID    float64
Country         object
dtype: object

## Step 2: Calculating Cohort Index

In [13]:
retail['InvoiceDate'] = pd.to_datetime(retail['InvoiceDate'])

In [14]:
retail['YearMonth'] = retail['InvoiceDate'].dt.strftime("%Y-%m")
retail['YearMonth'] = pd.to_datetime(retail['YearMonth'])

In [15]:
retail['CohortMonth'] = retail.groupby(by=['Customer ID'])['YearMonth'].transform('min')

In [20]:
def calculate_cohort_index(df, transactions_col, cohort_month):
    
    year_diff = df[transactions_col].dt.year - df[cohort_month].dt.year

    month_diff = df[transactions_col].dt.month - df[cohort_month].dt.month
    
    cohort_index = year_diff * 12 + month_diff + 1
    
    return cohort_index

In [21]:
retail['CohortIndex'] = calculate_cohort_index(retail, 'InvoiceDate', 'CohortMonth')

In [23]:
retail['CohortIndex'].max()

25

In [25]:
retail.tail()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,YearMonth,CohortMonth,CohortIndex
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,2011-12-01,2011-08-01,5
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-01,2011-08-01,5
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,2011-12-01,2011-08-01,5
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France,2011-12-01,2011-08-01,5
1067370,581587,POST,POSTAGE,1,2011-12-09 12:50:00,18.00,12680.0,France,2011-12-01,2011-08-01,5


## Step 3: Calculating Average Active Users per Cohort

In [32]:
users_in_cohorts = retail.groupby(by=['CohortMonth', 'CohortIndex'])['Customer ID'].nunique()

In [34]:
users_in_cohorts = users_in_cohorts.reset_index()
users_in_cohorts.head()

,CohortMonth,CohortIndex,Customer ID
0,2009-12-01,1,1045
1,2009-12-01,2,392
2,2009-12-01,3,358
3,2009-12-01,4,447
4,2009-12-01,5,410


In [36]:
cohorts_fin = users_in_cohorts.pivot(index='CohortMonth', columns='CohortIndex', values='Customer ID')
cohorts_fin

CohortIndex,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
CohortMonth,,,,,,,,,,,,,,,,,,,,,
2009-12-01,1045.0,392.0,358.0,447.0,410.0,408.0,408.0,374.0,355.0,392.0,...,319.0,273.0,316.0,303.0,287.0,274.0,332.0,319.0,427.0,218.0
2010-01-01,394.0,86.0,119.0,120.0,110.0,115.0,105.0,91.0,114.0,134.0,...,60.0,86.0,74.0,69.0,73.0,93.0,73.0,88.0,29.0,NaN
2010-02-01,363.0,109.0,82.0,110.0,93.0,76.0,79.0,103.0,100.0,106.0,...,74.0,67.0,61.0,53.0,85.0,90.0,62.0,23.0,NaN,NaN
2010-03-01,436.0,95.0,113.0,103.0,100.0,87.0,105.0,130.0,126.0,50.0,...,74.0,76.0,69.0,74.0,89.0,93.0,33.0,NaN,NaN,NaN
2010-04-01,291.0,67.0,58.0,47.0,54.0,67.0,79.0,76.0,33.0,34.0,...,43.0,41.0,41.0,50.0,61.0,19.0,NaN,NaN,NaN,NaN
2010-05-01,254.0,49.0,45.0,49.0,48.0,66.0,56.0,33.0,17.0,22.0,...,33.0,36.0,42.0,40.0,12.0,NaN,NaN,NaN,NaN,NaN
2010-06-01,269.0,58.0,53.0,55.0,62.0,76.0,35.0,25.0,22.0,32.0,...,33.0,37.0,55.0,16.0,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,183.0,38.0,37.0,52.0,55.0,28.0,21.0,28.0,26.0,22.0,...,32.0,45.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,158.0,39.0,50.0,51.0,29.0,21.0,16.0,22.0,23.0,21.0,...,32.0,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 4: Calculating Retention Rates

In [48]:
cohort_sizes = cohorts_fin.iloc[:, 0]
retention = cohorts_fin.divide(cohort_sizes, axis='rows').round(3)

In [49]:
retention

CohortIndex,1,2,3,4,5,6,7,8,9,10,...,16,17,18,19,20,21,22,23,24,25
CohortMonth,,,,,,,,,,,,,,,,,,,,,
2009-12-01,1.0,0.375,0.343,0.428,0.392,0.390,0.390,0.358,0.340,0.375,...,0.305,0.261,0.302,0.290,0.275,0.262,0.318,0.305,0.409,0.209
2010-01-01,1.0,0.218,0.302,0.305,0.279,0.292,0.266,0.231,0.289,0.340,...,0.152,0.218,0.188,0.175,0.185,0.236,0.185,0.223,0.074,NaN
2010-02-01,1.0,0.300,0.226,0.303,0.256,0.209,0.218,0.284,0.275,0.292,...,0.204,0.185,0.168,0.146,0.234,0.248,0.171,0.063,NaN,NaN
2010-03-01,1.0,0.218,0.259,0.236,0.229,0.200,0.241,0.298,0.289,0.115,...,0.170,0.174,0.158,0.170,0.204,0.213,0.076,NaN,NaN,NaN
2010-04-01,1.0,0.230,0.199,0.162,0.186,0.230,0.271,0.261,0.113,0.117,...,0.148,0.141,0.141,0.172,0.210,0.065,NaN,NaN,NaN,NaN
2010-05-01,1.0,0.193,0.177,0.193,0.189,0.260,0.220,0.130,0.067,0.087,...,0.130,0.142,0.165,0.157,0.047,NaN,NaN,NaN,NaN,NaN
2010-06-01,1.0,0.216,0.197,0.204,0.230,0.283,0.130,0.093,0.082,0.119,...,0.123,0.138,0.204,0.059,NaN,NaN,NaN,NaN,NaN,NaN
2010-07-01,1.0,0.208,0.202,0.284,0.301,0.153,0.115,0.153,0.142,0.120,...,0.175,0.246,0.093,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-08-01,1.0,0.247,0.316,0.323,0.184,0.133,0.101,0.139,0.146,0.133,...,0.203,0.070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Step 5: Calculate Basic CLV (revenue-based)

**Basic CLV** = Average Revenue * Average Lifespan

In [54]:
retail['PurchaseSize'] = retail['Quantity']*retail['Price']

In [158]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

603.4894708998612

Average Lifespan can be inferred by looking into the average time it takes for customers to churn from the time they made their first purchase. For now, we assume that the customer lifespan is 3 years or 36 months.

In [155]:
lifespan_months = 36

In [159]:
CLV_basic = monthly_revenue * lifespan_months
CLV_basic

21725.620952395002

Basic Customer Lifetime Value is 21725.62 USD.

## Step 6: Calculate Granular CLV (revenue-based)

**Granular CLV** = (Average Revenue per Transaction * Average Frequency) * Average Lifespan

In [145]:
# first mean() averages the revenue per invoice 
# second mean() averages the revenue per purchase
revenue_per_purchase = retail.groupby(by=['Invoice'])['PurchaseSize'].mean().mean()
revenue_per_purchase

31.16675777794161

In [152]:
# calculate average number of unique invoices per customer each month
avg_frequency_per_month = retail.groupby(by=['Customer ID', 'YearMonth'])['Invoice'].nunique().mean()
avg_frequency_per_month

1.6625050939132369

In [153]:
CLV_granular = revenue_per_purchase * avg_frequency_per_month * lifespan_months
CLV_granular

1865.336168397165

Granular CLV is 1865.34 USD, with each customer making on average 1.66 transactions per month and spending about 31 USD per transaction.

## Step 7: Calculate Traditional CLV (revenue-based)

Traditional CLV method doens't require to define customer lifespan and instead uses retention to churn rate to access customer life expectancy.

**Traditional CLV** = Average Revenue * Retention Rate / Churn Rate

In [162]:
# calculate monthly spend per customer
monthly_revenue = retail.groupby(by=['Customer ID', 'YearMonth'])['PurchaseSize'].sum().mean()
monthly_revenue

603.4894708998612

In [169]:
#calculate average retention rate per customer
avg_retention = retention.iloc[:, 1:].mean().mean()
avg_retention

0.1901439413052479

In [171]:
avg_churn = 1 - avg_retention
avg_churn

0.8098560586947521

In [176]:
CLV_traditional = monthly_revenue * (avg_retention/avg_churn)
CLV_traditional

141.69168125760655

In [187]:
print("Average CLV: {:.2f} USD at {:.1%} average retention rate and {:.2f} USD average monthly revenue".format(CLV_traditional, avg_retention, monthly_revenue))

Average CLV: 141.69 USD at 19.0% average retention rate and 603.49 USD average monthly revenue


## Conclusion

We calculated three types Lifetime Customer Values - Basic, Granular and Traditional. Basic LCV gave unrealistically high LCV - 21725.62 USD per customer. Granular CLV is much lower - with only 1865.33 USD per customer. Still, both Basic and Traditional CLV relied on an arbitrary value of lifespan per customer, which we assumed here to be 3 years. <br>
The Traditional CLV, however, gave a more realistical number - only 141.69 USD per customer and was based on the real retention to churn ratio as a proxy for the customer lifespan. <br>
Still, the traditional CLV method assumed that the churn is final, i.e. customers that churn do not come back later. Hence, it might underreport actual CLV, especially with low retention rates as in our case (19%).

## Acknowledgement

**Data Source:** Online Retail II Data Set, UCI Machine Learning Repository, http://archive.ics.uci.edu/ml/datasets/Online+Retail+II

Analysis done after completing the course "Customer Segmentation in Python" on DataCamp (instructor - 
Karolis Urbonas)

